In [ ]:
!pip install stitching

In [ ]:
import zipfile

def unzip_file(zip_file_path, extract_to_path):
  """Unzips a ZIP file.

  Args:
    zip_file_path: Path to the ZIP file.
    extract_to_path: Path to the directory where the files will be extracted.
  """
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

# Example usage
zip_file_path = '/content/Poombuhar Real Data Set.zip'  # Replace with the path to your ZIP file
extract_to_path = '/content/extracted_files'  # Replace with the desired extraction path
unzip_file(zip_file_path, extract_to_path)

In [ ]:
from stitching import Stitcher
stitcher = Stitcher()

In [ ]:
import cv2
import numpy as np
import os

def preprocess_underwater_image(image_path, output_path):
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Cannot read image: {image_path}")

    # Convert to LAB color space for better color processing
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Enhance contrast of luminance channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l = clahe.apply(l)

    # Color correction
    a = cv2.subtract(a, 5)  # Reduce green tint
    b = cv2.subtract(b, 5)  # Reduce blue tint

    # Merge channels
    enhanced_lab = cv2.merge((l, a, b))
    enhanced = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)

    # Denoise
    denoised = cv2.fastNlMeansDenoisingColored(enhanced, None, 10, 10, 7, 21)

    # Reduce resolution
    height, width = denoised.shape[:2]
    new_height = 720  # Standard HD height
    new_width = int(width * (new_height / height))
    resized = cv2.resize(denoised, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # Save result
    cv2.imwrite(output_path, resized)
    return resized

def process_folder(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"processed_{filename}")
            try:
                preprocess_underwater_image(input_path, output_path)
                print(f"Processed: {filename}")
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")

# Usage
input_folder = "/content/extracted_files/Poombuhar Real Data Set/Batch 2"
output_folder = "/content/extracted_files/Poombuhar Real Data Set/processed_images"
process_folder(input_folder, output_folder)

Processed: down_frame00015.png
Processed: down_frame00000.png
Processed: down_frame00018.png
Processed: down_frame00040.png
Processed: down_frame00039.png
Processed: down_frame00022.png
Processed: down_frame00007.png
Processed: down_frame00009.png
Processed: down_frame00035.png
Processed: down_frame00048.png
Processed: down_frame00019.png
Processed: down_frame00030.png
Processed: down_frame00013.png
Processed: down_frame00003.png
Processed: down_frame00002.png
Processed: down_frame00023.png
Processed: down_frame00041.png
Processed: down_frame00029.png
Processed: down_frame00044.png
Processed: down_frame00010.png
Processed: down_frame00014.png
Processed: down_frame00025.png
Processed: down_frame00017.png
Processed: down_frame00020.png
Processed: down_frame00038.png
Processed: down_frame00036.png
Processed: down_frame00042.png
Processed: down_frame00011.png
Processed: down_frame00016.png
Processed: down_frame00008.png
Processed: down_frame00047.png
Processed: down_frame00032.png
Processe

In [ ]:
# import os
# import cv2
# import numpy as np
# from stitching import Stitcher
# import shutil

# # Define the path to the master folder and output folder
# master_folder_path = "/content/extracted_files/Poombuhar Real Data Set"
# stitched_images_folder = "/content/random"

# # Standard resolution for downscaling
# STANDARD_WIDTH = 1280
# STANDARD_HEIGHT = 720

# # Enhanced stitcher settings
# settings = {
#     "detector": "sift",
#     "confidence_threshold": 0.2,
#     "estimator": "homography",
#     "matches_graph_dot_file": "matches_graph.dot",
# }

# def enhance_image(image):
#     """Enhance image features to improve matching."""
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
#     enhanced = clahe.apply(gray)
#     return cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)

# def resize_to_standard(image_path, width, height):
#     """Resize and enhance image."""
#     image = cv2.imread(image_path)
#     if image is None:
#         raise ValueError(f"Failed to read image: {image_path}")

#     enhanced = enhance_image(image)

#     h, w = enhanced.shape[:2]
#     aspect = w / h
#     if aspect > width / height:
#         new_w = width
#         new_h = int(width / aspect)
#     else:
#         new_h = height
#         new_w = int(height * aspect)

#     resized = cv2.resize(enhanced, (new_w, new_h), interpolation=cv2.INTER_AREA)
#     return resized

# def try_stitch_with_different_params(images, attempts=3):
#     """Try stitching with different parameters."""
#     errors = []

#     # Attempt 1: Default parameters
#     try:
#         stitcher = Stitcher(**settings)
#         return stitcher.stitch(images)
#     except Exception as e:
#         errors.append(f"Default params failed: {str(e)}")

#     # Attempt 2: Lower confidence threshold
#     try:
#         modified_settings = settings.copy()
#         modified_settings["confidence_threshold"] = 0.1
#         stitcher = Stitcher(**modified_settings)
#         return stitcher.stitch(images)
#     except Exception as e:
#         errors.append(f"Lower confidence failed: {str(e)}")

#     # Attempt 3: Try with affine estimator
#     try:
#         modified_settings = settings.copy()
#         modified_settings["estimator"] = "affine"
#         stitcher = Stitcher(**modified_settings)
#         return stitcher.stitch(images)
#     except Exception as e:
#         errors.append(f"Affine estimator failed: {str(e)}")

#     raise Exception(f"All stitching attempts failed: {'; '.join(errors)}")

# def create_batch_folder(output_folder, batch_num, image_names):
#     """Create a folder for each batch and copy original images."""
#     batch_folder = os.path.join(output_folder, f"batch_{batch_num}")
#     originals_folder = os.path.join(batch_folder, "original_images")
#     os.makedirs(originals_folder, exist_ok=True)

#     # Copy original images to the batch folder
#     for img_path in image_names:
#         shutil.copy2(img_path, originals_folder)

#     return batch_folder

# def stitch_images_in_sliding_window(folder_path, output_folder, window_size=4):
#     """Stitch images using sliding window approach with better end case handling."""
#     images = [os.path.join(folder_path, img) for img in os.listdir(folder_path)
#               if img.lower().endswith(('.jpg', '.png', '.jpeg'))]
#     images.sort()

#     total_images = len(images)
#     total_full_batches = (total_images - window_size + 1)

#     # Process regular sliding windows
#     for i in range(total_full_batches):
#         batch = images[i:i + window_size]
#         print(f"Processing batch {i+1}: {[os.path.basename(img) for img in batch]}")

#         batch_folder = create_batch_folder(output_folder, i+1, batch)

#         try:
#             resized_batch = []
#             for img_path in batch:
#                 resized_image = resize_to_standard(img_path, STANDARD_WIDTH, STANDARD_HEIGHT)
#                 temp_path = os.path.join("/tmp", f"temp_{os.path.basename(img_path)}")
#                 cv2.imwrite(temp_path, resized_image)
#                 resized_batch.append(temp_path)

#             panorama = try_stitch_with_different_params(resized_batch)
#             output_path = os.path.join(batch_folder, "stitched_result.jpg")
#             cv2.imwrite(output_path, panorama)
#             print(f"Successfully stitched batch {i+1}")
#         except Exception as e:
#             print(f"Error stitching batch {i+1}: {e}")

#     # Handle remaining images
#     remaining_start = total_full_batches
#     remaining_images = images[remaining_start:]

#     if len(remaining_images) > 1:
#         print(f"Processing remaining {len(remaining_images)} images")

#         # Process remaining images in window_size chunks
#         for i in range(0, len(remaining_images) - 1, window_size):
#             end_idx = min(i + window_size, len(remaining_images))
#             current_batch = remaining_images[i:end_idx]

#             if len(current_batch) < 2:  # Skip if only one image left
#                 continue

#             batch_num = f"remaining_{remaining_start + i}"
#             print(f"Processing remaining batch: {[os.path.basename(img) for img in current_batch]}")

#             batch_folder = create_batch_folder(output_folder, batch_num, current_batch)

#             try:
#                 resized_batch = []
#                 for img_path in current_batch:
#                     resized_image = resize_to_standard(img_path, STANDARD_WIDTH, STANDARD_HEIGHT)
#                     temp_path = os.path.join("/tmp", f"temp_{os.path.basename(img_path)}")
#                     cv2.imwrite(temp_path, resized_image)
#                     resized_batch.append(temp_path)

#                 panorama = try_stitch_with_different_params(resized_batch)
#                 output_path = os.path.join(batch_folder, "stitched_result.jpg")
#                 cv2.imwrite(output_path, panorama)
#                 print(f"Successfully stitched remaining batch {batch_num}")
#             except Exception as e:
#                 print(f"Error stitching remaining batch {batch_num}: {e}")

#                 # If batch fails, try with smaller subsets
#                 if len(current_batch) > 2:
#                     for j in range(len(current_batch) - 1):
#                         subset = current_batch[j:j+2]
#                         subset_batch_num = f"remaining_{batch_num}_subset_{j}"
#                         print(f"Attempting subset: {[os.path.basename(img) for img in subset]}")

#                         try:
#                             resized_subset = []
#                             for img_path in subset:
#                                 resized_image = resize_to_standard(img_path, STANDARD_WIDTH, STANDARD_HEIGHT)
#                                 temp_path = os.path.join("/tmp", f"temp_{os.path.basename(img_path)}")
#                                 cv2.imwrite(temp_path, resized_image)
#                                 resized_subset.append(temp_path)

#                             subset_panorama = try_stitch_with_different_params(resized_subset)
#                             subset_folder = create_batch_folder(output_folder, subset_batch_num, subset)
#                             output_path = os.path.join(subset_folder, "stitched_result.jpg")
#                             cv2.imwrite(output_path, subset_panorama)
#                             print(f"Successfully stitched subset {subset_batch_num}")
#                         except Exception as e:
#                             print(f"Error stitching subset {subset_batch_num}: {e}")
# def process_folders(master_folder, output_folder, window_size=4):
#     """Process all folders with enhanced organization."""
#     os.makedirs(output_folder, exist_ok=True)

#     for folder_name in os.listdir(master_folder):
#         folder_path = os.path.join(master_folder, folder_name)
#         if os.path.isdir(folder_path):
#             print(f"\nProcessing folder: {folder_name}")
#             folder_output_path = os.path.join(output_folder, folder_name)
#             os.makedirs(folder_output_path, exist_ok=True)

#             try:
#                 stitch_images_in_sliding_window(folder_path, folder_output_path, window_size)
#                 print(f"Successfully processed {folder_name}")
#             except Exception as e:
#                 print(f"Error processing folder {folder_name}: {e}")

# # Run the processing
# window_size = 4
# process_folders(master_folder_path, stitched_images_folder, window_size)

In [ ]:
import os
import cv2
import numpy as np
from stitching import Stitcher
import shutil

# Define the path to the master folder and output folder
master_folder_path = "/content/extracted_files/Poombuhar Real Data Set"
stitched_images_folder = "/content/random"

# Standard resolution for downscaling
STANDARD_WIDTH = 1280
STANDARD_HEIGHT = 720

# Group sizes for stitching
GROUP_SIZES = [2, 4, 8, 16]

# Enhanced stitcher settings
settings = {
    "detector": "sift",
    "confidence_threshold": 0.2,
    "estimator": "homography",
    "matches_graph_dot_file": "matches_graph.dot",
}

def enhance_image(image):
    """Enhance image features to improve matching."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    return cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)

def resize_to_standard(image_path, width, height):
    """Resize and enhance image."""
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to read image: {image_path}")

    enhanced = enhance_image(image)

    h, w = enhanced.shape[:2]
    aspect = w / h
    if aspect > width / height:
        new_w = width
        new_h = int(width / aspect)
    else:
        new_h = height
        new_w = int(height * aspect)

    resized = cv2.resize(enhanced, (new_w, new_h), interpolation=cv2.INTER_AREA)
    return resized

def try_stitch_with_different_params(images, attempts=3):
    """Try stitching with different parameters."""
    errors = []

    # Attempt 1: Default parameters
    try:
        stitcher = Stitcher(**settings)
        return stitcher.stitch(images)
    except Exception as e:
        errors.append(f"Default params failed: {str(e)}")

    # Attempt 2: Lower confidence threshold
    try:
        modified_settings = settings.copy()
        modified_settings["confidence_threshold"] = 0.1
        stitcher = Stitcher(**modified_settings)
        return stitcher.stitch(images)
    except Exception as e:
        errors.append(f"Lower confidence failed: {str(e)}")

    # Attempt 3: Try with affine estimator
    try:
        modified_settings = settings.copy()
        modified_settings["estimator"] = "affine"
        stitcher = Stitcher(**modified_settings)
        return stitcher.stitch(images)
    except Exception as e:
        errors.append(f"Affine estimator failed: {str(e)}")

    raise Exception(f"All stitching attempts failed: {'; '.join(errors)}")

def create_batch_folder(output_folder, group_size, batch_num, image_names):
    """Create a folder for each batch within the grouping folder."""
    group_folder = os.path.join(output_folder, f"grouping_{group_size}")
    batch_folder = os.path.join(group_folder, f"batch_{batch_num}")
    originals_folder = os.path.join(batch_folder, "original_images")

    os.makedirs(group_folder, exist_ok=True)
    os.makedirs(originals_folder, exist_ok=True)

    # Copy original images to the batch folder
    for img_path in image_names:
        shutil.copy2(img_path, originals_folder)

    return batch_folder

def stitch_image_group(images, output_folder, group_size, batch_num):
    """Stitch a group of images and save the result."""
    if len(images) < 2:
        return

    batch_folder = create_batch_folder(output_folder, group_size, batch_num, images)

    try:
        resized_batch = []
        for img_path in images:
            resized_image = resize_to_standard(img_path, STANDARD_WIDTH, STANDARD_HEIGHT)
            temp_path = os.path.join("/tmp", f"temp_{os.path.basename(img_path)}")
            cv2.imwrite(temp_path, resized_image)
            resized_batch.append(temp_path)

        panorama = try_stitch_with_different_params(resized_batch)
        output_path = os.path.join(batch_folder, "stitched_result.jpg")
        cv2.imwrite(output_path, panorama)
        print(f"Successfully stitched group size {group_size}, batch {batch_num}")
    except Exception as e:
        print(f"Error stitching group size {group_size}, batch {batch_num}: {e}")

def stitch_images_with_multiple_groups(folder_path, output_folder):
    """Stitch images using different group sizes."""
    images = [os.path.join(folder_path, img) for img in os.listdir(folder_path)
              if img.lower().endswith(('.jpg', '.png', '.jpeg'))]
    images.sort()

    for group_size in GROUP_SIZES:
        print(f"\nProcessing group size: {group_size}")

        # Process images in groups of the current size
        for i in range(0, len(images), group_size):
            current_group = images[i:i + group_size]
            if len(current_group) < 2:  # Skip if only one image left
                continue

            batch_num = i // group_size + 1
            print(f"Processing batch {batch_num} with {len(current_group)} images")
            stitch_image_group(current_group, output_folder, group_size, batch_num)

def process_folders(master_folder, output_folder):
    """Process all folders with different group sizes."""
    os.makedirs(output_folder, exist_ok=True)

    for folder_name in os.listdir(master_folder):
        folder_path = os.path.join(master_folder, folder_name)
        if os.path.isdir(folder_path):
            print(f"\nProcessing folder: {folder_name}")
            folder_output_path = os.path.join(output_folder, folder_name)
            os.makedirs(folder_output_path, exist_ok=True)

            try:
                stitch_images_with_multiple_groups(folder_path, folder_output_path)
                print(f"Successfully processed {folder_name}")
            except Exception as e:
                print(f"Error processing folder {folder_name}: {e}")

# Run the processing
process_folders(master_folder_path, stitched_images_folder)


Processing folder: Batch 3

Processing group size: 2
Processing batch 1 with 2 images
Successfully stitched group size 2, batch 1
Processing batch 2 with 2 images
Successfully stitched group size 2, batch 2
Processing batch 3 with 2 images
Successfully stitched group size 2, batch 3
Processing batch 4 with 2 images
Successfully stitched group size 2, batch 4
Processing batch 5 with 2 images
Successfully stitched group size 2, batch 5
Processing batch 6 with 2 images
Successfully stitched group size 2, batch 6
Processing batch 7 with 2 images
Successfully stitched group size 2, batch 7
Processing batch 8 with 2 images
Successfully stitched group size 2, batch 8
Processing batch 9 with 2 images
Successfully stitched group size 2, batch 9
Processing batch 10 with 2 images
Successfully stitched group size 2, batch 10
Processing batch 11 with 2 images
Successfully stitched group size 2, batch 11
Processing batch 12 with 2 images
Successfully stitched group size 2, batch 12
Processing batch

In [ ]:
import os
import zipfile
from datetime import datetime

def zip_folder_structure(source_folder, output_zip=None):
    """
    Zip an entire folder structure, maintaining all subdirectories and their contents.

    Args:
        source_folder (str): Path to the folder to be zipped
        output_zip (str): Optional custom path for the output zip file
                         If not provided, creates a timestamped zip in the same directory

    Returns:
        str: Path to the created zip file
    """
    # If no output path specified, create one with timestamp
    if output_zip is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        base_folder_name = os.path.basename(source_folder)
        output_zip = f"{source_folder}_{timestamp}.zip"

    # Normalize paths
    source_folder = os.path.abspath("/content/random")
    output_zip = os.path.abspath(output_zip)

    try:
        with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # Walk through all directories and files
            for root, dirs, files in os.walk(source_folder):
                # Calculate relative path from source folder
                relative_path = os.path.relpath(root, source_folder)

                # Add empty directories
                if not files and not dirs:
                    # Add directory itself (empty)
                    zip_path = os.path.join(relative_path, '')
                    zipf.write(root, zip_path)

                # Add all files in current directory
                for file in files:
                    file_path = os.path.join(root, file)
                    zip_path = os.path.join(relative_path, file)

                    try:
                        zipf.write(file_path, zip_path)
                        print(f"Added: {zip_path}")
                    except Exception as e:
                        print(f"Error adding {file_path}: {str(e)}")
                        continue

        print(f"\nZip file created successfully at: {output_zip}")
        print(f"Size: {os.path.getsize(output_zip) / (1024*1024):.2f} MB")
        return output_zip

    except Exception as e:
        print(f"Error creating zip file: {str(e)}")
        return None

# Example usage with the stitched images folder
def zip_stitched_results(stitched_images_folder):
    """
    Zip the entire stitched images folder after processing.

    Args:
        stitched_images_folder (str): Path to the folder containing stitched results
    """
    # Create a timestamped zip file
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    zip_name = f"stitched_results_{timestamp}.zip"
    zip_path = os.path.join(os.path.dirname(stitched_images_folder), zip_name)

    return zip_folder_structure(stitched_images_folder, zip_path)

# Add this to the end of your main processing code
if __name__ == "__main__":
    # After processing all images
    print("\nCreating zip archive of results...")
    zip_file = zip_stitched_results(stitched_images_folder)
    if zip_file:
        print(f"Results archived in: {zip_file}")
    else:
        print("Failed to create zip archive")